In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, rand
import time

# Generar un DataFrame grande de prueba con datos aleatorios
# Aproximadamente 10 millones de filas
df = spark.range(0, 10_000_000).withColumn("valor", (rand() * 1000).cast("int"))

# Simular una carga de trabajo que genere múltiples stages y dure unos minutos
# Paso 1: Aplicar transformaciones que generen un shuffle (como groupBy)
df_filtrado = df.filter(col("valor") > 100)
df_transformado = df_filtrado.withColumn("categoria", col("valor") % 10)
df_agrupado = df_transformado.groupBy("categoria").count()

# Paso 2: Unión con otro DataFrame para forzar otra etapa
df_extra = spark.range(0, 10).withColumnRenamed("id", "categoria")
df_final = df_agrupado.join(df_extra, on="categoria", how="inner")

# Paso 3: Escritura en memoria para forzar ejecución
df_final.persist()
df_final.count()

# Pausa artificial para mantener la aplicación activa y permitir visualización del DAG
print("Esperando 5 minutos para observación del DAG en Spark UI...")
time.sleep(300)
